# Tutorial 06: Plan and Execute

In this tutorial, you'll learn how to build agents that plan before they act, breaking complex tasks into manageable steps.

**What you'll learn:**
- **Planning**: Breaking tasks into steps
- **Execution**: Processing steps sequentially
- **Re-planning**: Adjusting plans based on results
- **Structured outputs**: Using Pydantic for plans

By the end, you'll have an agent that plans, executes, and adapts.

## Why Plan and Execute?

ReAct agents (Tutorial 02) decide step-by-step. This works well for simple tasks but struggles with:
- **Multi-step problems**: Need to think ahead
- **Resource efficiency**: Planning can use a stronger model, execution a faster one
- **Visibility**: Explicit plans are easier to review and debug

### The Pattern

1. **Plan**: Create a list of steps to accomplish the goal
2. **Execute**: Work through steps one at a time
3. **Re-plan**: Update the plan if needed based on results

## Graph Visualization

![Plan and Execute Graph](../docs/images/06-plan-execute-graph.png)

The planner creates a plan, the executor processes each step, and optionally re-plans.

In [ ]:
# Setup
from langgraph_ollama_local import LocalAgentConfig

config = LocalAgentConfig()
print(f"Ollama: {config.ollama.base_url}")
print(f"Model: {config.ollama.model}")

## Step 1: Define State

Our state tracks the task, plan, and execution progress:

In [ ]:
from typing import Annotated, List, Tuple
import operator
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class PlanExecuteState(TypedDict):
    """State for plan-and-execute agent."""
    messages: Annotated[list, add_messages]  # Conversation history
    task: str                                 # Original task
    plan: List[str]                           # List of steps
    current_step: int                         # Current step index
    past_steps: Annotated[List[Tuple[str, str]], operator.add]  # (step, result) pairs
    response: str                             # Final response

print("State defined with: task, plan, current_step, past_steps, response")

## Step 2: Create the LLM

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0,
)

print("LLM configured")

## Step 3: Create the Planner Node

The planner breaks down the task into steps:

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
import json
import re

PLANNER_PROMPT = """You are a task planner. Break down the given task into clear, actionable steps.

Return your plan as a JSON array of strings, where each string is one step.
Keep steps simple and focused. Aim for 3-5 steps.

Example output:
["Step 1: Research the topic", "Step 2: Outline main points", "Step 3: Write the content"]

Return ONLY the JSON array, no other text.
"""

def planner_node(state: PlanExecuteState) -> dict:
    """Create a plan for the task."""
    task = state["task"]
    
    messages = [
        SystemMessage(content=PLANNER_PROMPT),
        HumanMessage(content=f"Create a plan for: {task}")
    ]
    
    response = llm.invoke(messages)
    content = response.content
    
    # Parse JSON from response
    try:
        # Try to find JSON array in response
        match = re.search(r'\[.*?\]', content, re.DOTALL)
        if match:
            plan = json.loads(match.group())
        else:
            # Fallback: split by newlines
            plan = [line.strip() for line in content.split('\n') if line.strip()]
    except json.JSONDecodeError:
        plan = [content]  # Use whole response as single step
    
    print(f"\n=== PLAN ===")
    for i, step in enumerate(plan):
        print(f"  {i+1}. {step}")
    
    return {
        "plan": plan,
        "current_step": 0
    }

## Step 4: Create the Executor Node

The executor processes one step at a time:

In [ ]:
EXECUTOR_PROMPT = """You are a task executor. Complete the given step as part of a larger task.

Provide a clear, concise result for this step.
Focus on actionable output that helps accomplish the overall task.
"""

def executor_node(state: PlanExecuteState) -> dict:
    """Execute the current step."""
    plan = state["plan"]
    current_step = state["current_step"]
    task = state["task"]
    past_steps = state.get("past_steps", [])
    
    if current_step >= len(plan):
        return {}  # No more steps
    
    step = plan[current_step]
    
    # Build context from past steps
    context = f"Original task: {task}\n\n"
    if past_steps:
        context += "Previous steps completed:\n"
        for prev_step, prev_result in past_steps:
            context += f"- {prev_step}: {prev_result[:100]}...\n"
        context += "\n"
    
    messages = [
        SystemMessage(content=EXECUTOR_PROMPT),
        HumanMessage(content=f"{context}Now execute this step: {step}")
    ]
    
    response = llm.invoke(messages)
    result = response.content
    
    print(f"\n=== STEP {current_step + 1} ===")
    print(f"  Executing: {step}")
    print(f"  Result: {result[:200]}..." if len(result) > 200 else f"  Result: {result}")
    
    return {
        "past_steps": [(step, result)],
        "current_step": current_step + 1
    }

## Step 5: Create the Finalizer Node

Combines all step results into a final response:

In [ ]:
FINALIZER_PROMPT = """You are a task finalizer. Combine the results of all completed steps into a coherent final response.

Synthesize the information and provide a complete answer to the original task.
"""

def finalizer_node(state: PlanExecuteState) -> dict:
    """Create final response from all step results."""
    task = state["task"]
    past_steps = state.get("past_steps", [])
    
    # Build summary of all steps
    steps_summary = "\n".join([
        f"Step: {step}\nResult: {result}"
        for step, result in past_steps
    ])
    
    messages = [
        SystemMessage(content=FINALIZER_PROMPT),
        HumanMessage(content=f"Original task: {task}\n\nCompleted steps:\n{steps_summary}\n\nProvide the final response.")
    ]
    
    response = llm.invoke(messages)
    
    print(f"\n=== FINAL RESPONSE ===")
    print(response.content)
    
    return {"response": response.content}

## Step 6: Define Routing Logic

In [ ]:
def should_continue(state: PlanExecuteState) -> str:
    """Decide whether to execute more steps or finalize."""
    current_step = state["current_step"]
    plan = state["plan"]
    
    if current_step < len(plan):
        return "executor"  # More steps to execute
    return "finalizer"  # All steps done, finalize

print("Routing logic defined")

## Step 7: Build the Graph

In [ ]:
from langgraph.graph import StateGraph, START, END

# Build the plan-and-execute graph
workflow = StateGraph(PlanExecuteState)

# Add nodes
workflow.add_node("planner", planner_node)
workflow.add_node("executor", executor_node)
workflow.add_node("finalizer", finalizer_node)

# Add edges
workflow.add_edge(START, "planner")           # Start with planning
workflow.add_edge("planner", "executor")      # Then execute first step
workflow.add_conditional_edges(
    "executor",
    should_continue,
    {"executor": "executor", "finalizer": "finalizer"}
)
workflow.add_edge("finalizer", END)

# Compile
graph = workflow.compile()

print("Plan-and-execute graph compiled!")

In [ ]:
# Visualize
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not render: {e}")

## Step 8: Run the Agent

In [ ]:
# Run the plan-and-execute agent
task = "Explain the key benefits of using LangGraph for building AI agents, including at least 3 specific advantages."

print(f"Task: {task}")
print("="*60)

result = graph.invoke({
    "task": task,
    "messages": [],
    "plan": [],
    "current_step": 0,
    "past_steps": [],
    "response": ""
})

print("\n" + "="*60)
print("EXECUTION COMPLETE")
print(f"Steps executed: {len(result['past_steps'])}")

In [ ]:
# Helper function for easy use
def plan_and_execute(task: str) -> str:
    """Run plan-and-execute on a task."""
    result = graph.invoke({
        "task": task,
        "messages": [],
        "plan": [],
        "current_step": 0,
        "past_steps": [],
        "response": ""
    })
    return result["response"]

# Test with another task
response = plan_and_execute("Write a haiku about programming, then explain what each line means.")
print("\n" + "="*60)
print("Final response:")
print(response)

## Complete Code

In [ ]:
# Complete Plan-and-Execute Agent

import json
import re
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph_ollama_local import LocalAgentConfig

# === State ===
class PlanExecuteState(TypedDict):
    messages: Annotated[list, add_messages]
    task: str
    plan: List[str]
    current_step: int
    past_steps: Annotated[List[Tuple[str, str]], operator.add]
    response: str

# === LLM ===
config = LocalAgentConfig()
llm = ChatOllama(model=config.ollama.model, base_url=config.ollama.base_url, temperature=0)

# === Nodes ===
def planner(state: PlanExecuteState) -> dict:
    response = llm.invoke([HumanMessage(content=f"Break this into 3-5 steps (JSON array): {state['task']}")])
    try:
        plan = json.loads(re.search(r'\[.*?\]', response.content, re.DOTALL).group())
    except:
        plan = [response.content]
    return {"plan": plan, "current_step": 0}

def executor(state: PlanExecuteState) -> dict:
    if state["current_step"] >= len(state["plan"]):
        return {}
    step = state["plan"][state["current_step"]]
    response = llm.invoke([HumanMessage(content=f"Execute: {step}")])
    return {"past_steps": [(step, response.content)], "current_step": state["current_step"] + 1}

def finalizer(state: PlanExecuteState) -> dict:
    summary = "\n".join([f"{s}: {r}" for s, r in state["past_steps"]])
    response = llm.invoke([HumanMessage(content=f"Summarize results for '{state['task']}':\n{summary}")])
    return {"response": response.content}

def should_continue(state: PlanExecuteState) -> str:
    return "executor" if state["current_step"] < len(state["plan"]) else "finalizer"

# === Graph ===
workflow = StateGraph(PlanExecuteState)
workflow.add_node("planner", planner)
workflow.add_node("executor", executor)
workflow.add_node("finalizer", finalizer)
workflow.add_edge(START, "planner")
workflow.add_edge("planner", "executor")
workflow.add_conditional_edges("executor", should_continue, {"executor": "executor", "finalizer": "finalizer"})
workflow.add_edge("finalizer", END)
graph = workflow.compile()

# === Use ===
result = graph.invoke({"task": "List 3 benefits of Python", "messages": [], "plan": [], "current_step": 0, "past_steps": [], "response": ""})
print(result["response"])

## Key Concepts Recap

| Concept | Description |
|---------|-------------|
| **Planner** | Creates a list of steps from the task |
| **Executor** | Processes steps one at a time |
| **Finalizer** | Combines results into final response |
| **past_steps** | Accumulates (step, result) pairs |
| **current_step** | Tracks progress through plan |

## Advantages

1. **Explicit reasoning**: Plan is visible and debuggable
2. **Resource efficiency**: Use different models for planning vs execution
3. **Flexibility**: Easy to add re-planning based on results

## What's Next?

In [Tutorial 07: Research Assistant](07_research_assistant.ipynb), you'll combine all patterns into a comprehensive research agent.